In [ ]:
from scipy.io import mmread
import xb.formatting as xf
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import os
from xb.formatting import keep_nuclei_and_quality
import xb.plotting as xp
from xb.plotting import map_of_clusters
import scipy as sp
from xb.calculating import domainassign
import seaborn as sns
import gzip
import shutil
import json
import matplotlib

You can export the Anndata object from the '11pcw_example_code' notebook once the cell type annotations are done. Those Anndata objects will be used for domain calculations.

## functions for banksy 

In [ ]:
## modify the X coords of each sample so that they are processed independently
def adapt_banksy_for_multisample(adata):
    adata.obs['Xres']=adata.obs['x_centroid']
    adata.obs['Yres']=adata.obs['y_centroid']
    gap=adata.obs['x_centroid'].max()+int(adata.obs['x_centroid'].max()/10)
    samplekey='sample'
    counter=0
    sampsel=[]
    for s in adata.obs[samplekey].unique():
        sampsel.append(s)
        adata.obs.loc[~adata.obs[samplekey].isin(sampsel),'Xres']=adata.obs.loc[~adata.obs[samplekey].isin(sampsel),'Xres']+gap
        counter=counter+1
    adata.obs['Yres']=adata.obs['Yres']+1
    adata.obsm['spatial_sample_specific']=adata.obsm['spatial']
    adata.obsm['spatial']=np.array(adata.obs.loc[:,['Xres','Yres']])
    return adata
def define_palette(n_colors=50):
    from random import randint
    colorlist = []
    n = n_colors
    for i in range(n):
        colorlist.append('#%06X' % randint(0, 0xFFFFFF))
    return colorlist

def domains_by_banksy(adata,plot_path:str,banksy_params:dict):
    adata=adapt_banksy_for_multisample(adata)
    coord_keys = ('Xres', 'Yres','spatial')
    prev_clusters=[e for e in adata.obs.columns if clustering_params['clustering_alg'] in e]
    if len(prev_clusters)>0:
        annotation_key=prev_clusters[0]
    else:
        adata.obs['default_clustering']='0'
        adata.obs['default_clustering']=adata.obs['default_clustering'].astype('category')
        annotation_key='default_clustering'
        

    banksy_dict = initialize_banksy(
        adata,
        coord_keys,
        banksy_params['k_geom'],
        nbr_weight_decay=banksy_params['nbr_weight_decay'],
        max_m=banksy_params['max_m'],
        plt_edge_hist=True,
        plt_nbr_weights=True,
        plt_agf_angles=False,
        plt_theta=False)
    results_df = run_banksy_multiparam(
        adata,
        banksy_dict,
        banksy_params['lambda_list'], banksy_params['resolutions'],
        color_list = define_palette(n_colors=100), max_m = banksy_params['max_m'], filepath = plot_path,
        key = coord_keys, pca_dims = banksy_params['pca_dims'],
        annotation_key = annotation_key, max_labels = None,
        cluster_algorithm = banksy_params['cluster_algorithm'], match_labels = False, savefig = save, add_nonspatial = False,
        variance_balance = False,
    )
    adata_res=results_df.loc[results_df.index[0],'adata']
    res=adata_res.obs.loc[:,['unique_cell_id',results_df.index[0]]]
    res.columns=['unique_cell_id','spatial_domain']
    adata_res=results_df.loc[results_df.index[0],'adata']
    res=adata_res.obs.loc[:,['unique_cell_id',results_df.index[0]]]
    res.columns=['unique_cell_id','spatial_domain']
    id2domain=dict(zip(res['unique_cell_id'],res['spatial_domain']))
    adata.obs['banksy_domain']=adata.obs['unique_cell_id'].map(id2domain).astype(str)
    adata.obsm['spatial']=adata.obsm['spatial_sample_specific']
    return adata,adata_res

clustering_params={'normalization_target_sum':100,
'min_counts_x_cell':40,
'min_genes_x_cell':15,
'scale':False,
'clustering_alg':'louvain',
'resolutions':[0.2,0.5,1.1],
'n_neighbors':15,'umap_min_dist':0.1,
'n_pcs':0}
save=True

In [ ]:
import warnings
warnings.filterwarnings("ignore") 
import random
import scipy.sparse as sparse
from scipy.sparse import csr_matrix, issparse
current_directory=os.getcwd()
os.chdir('/home/sergio/Banksy_py')
from banksy.initialize_banksy import initialize_banksy
from banksy.run_banksy import run_banksy_multiparam
random_seed = 1234
np.random.seed(random_seed)
os.chdir(current_directory)

here you can choose the folder you have all 5 Anndata objects for all the weeks.

In [ ]:
folderpath='/home/sergio/Jnotebooks/HDLCA_codex/whole_datasets-20240510T124054Z-001/whole_datasets/'

In [ ]:
files=os.listdir(folderpath)
files=[f for f in files if '.h5ad' in f]
files=[f for f in files if not 'domains' in f]
for f in files:
    adata=sc.read(folderpath+f)
    adata.obs['x_centroid']=adata.obs['x']
    adata.obs['y_centroid']=adata.obs['y']
    adata.obs['sample']=f
    adata.obs['unique_cell_id']=adata.obs['cell_id']
    tac=f.split('.')[0]
    outipath=folderpath+tac
    if not os.path.exists(outipath):
        os.mkdir(outipath)
    '''Input File'''
    banksy_params={'resolutions':[.25], # clustering resolution for Leiden clustering
    'pca_dims':[20], # number of dimensions to keep after PCA
    'lambda_list':[.8],# lambda
    'k_geom':10, # 15 spatial neighbours
    'max_m':1, # use AGF
    'nbr_weight_decay':"scaled_gaussian", # can also be "reciprocal", "uniform" or "ranked"
    'cluster_algorithm':'leiden'}
    ###########
    plot_path='/home/sergio/Jnotebooks/HDLCA_codex/fetal_lung_neighborhood_analysis/'+outipath
    adata,adata_banksy=domains_by_banksy(adata,plot_path=plot_path,banksy_params=banksy_params)
    domains_counts=pd.DataFrame(adata.obs.groupby('banksy_domain').count()['cell_id'])
    domains_counts.columns=['total_counts']
    plt.bar(domains_counts.index,domains_counts['total_counts'])
    plt.xlabel('Domains')
    plt.ylabel('Total cells')
    ###########
    domains_counts.transpose().plot(kind='bar',stacked=True,width=0.9)
    plt.savefig(folderpath+'barplot_domains.png')
    ###########
    sc.tl.rank_genes_groups(adata, groupby='banksy_domain', method='wilcoxon')
    sc.pl.rank_genes_groups_dotplot(adata, n_genes=3, swap_axes=True)#,save='deg.pdf')
    ###########
    domains=pd.crosstab(adata.obs['annotation'],adata.obs['banksy_domain'])
    domains=domains.div(domains.sum(axis=0),axis=1)
    sns.clustermap(domains,cmap='Blues')
    plt.savefig(folderpath+'heatmap_domain_by_celltype.png')
    ###########
    sc.set_figure_params(facecolor="white", figsize=(8, 8))
    for s in adata.obs['sample'].unique():
            adatasub=adata[adata.obs['sample']==s]
            sc.pl.spatial(adatasub,color='banksy_domain',spot_size=30,show=False)
            plt.savefig(folderpath+'spatial_domains.png')
    adata.write(folderpath+f.split('.')[0]+'_domains.h5ad')

In [ ]:
sc.set_figure_params(dpi=200,figsize=None)
files=os.listdir(folderpath)
files=[f for f in files if '.h5ad' in f]
files=[f for f in files if  'domains' in f]
files=[f for f in files if not  '8_domains' in f]
allexp=[]
alladatas=[]
for f in files:
    adata=sc.read(folderpath+f)
    meanexp=pd.crosstab(adata.obs['annotation'],adata.obs['banksy_domain']).transpose()
    sc.pl.spatial(adata,spot_size=50,color='banksy_domain')
    meanexp.index=f[13:17]+'_'+meanexp.index.astype(str)
    adata.obs['pcw']=f[13:17]
    adata.obs['region']=f.split('.')[0][5:11]
    allexp.append(meanexp)
    alladatas.append(adata)

In [ ]:
common_adata=sc.concat(alladatas)
signature_exp=pd.concat(allexp).fillna(0)
signature_exp=signature_exp.div(signature_exp.sum(axis=1),axis=0)
sns.clustermap(signature_exp,figsize=(20,20))
plt.savefig(folderpath+'heatmap_domain_by_celltype_alltypes.png')

In [ ]:
corr_exp=pd.DataFrame(np.corrcoef(signature_exp),index=signature_exp.index,columns=signature_exp.index)
sns.clustermap(corr_exp,figsize=(30,35),vmin=0.4)
plt.savefig(folderpath+'correlation_between_domains.png')